In [1]:
#!/usr/bin/env python3
import os, sys
import numpy as np
import itertools
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import scipy.linalg
import scipy.sparse

from scipy.integrate import solve_ivp
from matplotlib.lines import Line2D


from dicke_methods import coherent_spin_state, spin_op_vec_mat_dicke
from correlator_methods import mat_zxy_to_pzm, vec_zxy_to_pzm, multiply_vecs, \
    get_deriv_op_vec, sandwich_deriv_op_vec, add_deriv_op_vecs, multiply_deriv_op_vecs, \
    deriv_op_vec_to_vals, deriv_vals_to_correlators

from time import time

np.set_printoptions(linewidth = 200)

In [2]:
vecs_A = {}
vecs_B = {}
vecs_At_B = {}
vecs_B_At = {}
vecs_At_B_comm = {}
vecs_At_B_comm_sqr = {}
corrs_A = {}
corrs_B = {}
corrs_At_B = {}
corrs_B_At = {}
corrs_At_B_disc = {}
corrs_B_At_disc = {}
all_dicts = ( vecs_A, vecs_B, vecs_At_B, vecs_B_At, vecs_At_B_comm, vecs_At_B_comm_sqr,
              corrs_A, corrs_B, corrs_At_B, corrs_B_At, corrs_At_B_disc, corrs_B_At_disc )

vecs_A_txt = "vecs_A"
vecs_B_txt = "vecs_B"
vecs_At_B_txt = "vecs_At_B"
vecs_B_At_txt = "vecs_B_At"
vecs_At_B_comm_txt = "vecs_At_B_comm_txt"
vecs_At_B_comm_sqr_txt = "vecs_At_B_comm_sqr"
dicke = "dicke"
no_dec = "no_dec"
dec_M = "dec_M"
dec_m = "dec_m"
dec_P = "dec_P"
dec_p = "dec_p"
dec_Z = "dec_Z"
dec_z = "dec_z"
NAT = "NAT"
OAT = "OAT"
TAT = "TAT"
TNT = "TNT"
TNB = "TNB"
all_methods = [ NAT, OAT, TAT, TNT, TNB ]

save_dir = "../data/multi_time/"
fig_dir = "../figures/"

In [3]:
N = 10**4 # number of spins
order_cap = 35 # order limit for short-time correlator expansion
methods = [ OAT, TAT, TNT ]

def save_file_name(base_name, dec, method):
    return save_dir + f"{base_name}_{dec}_{method}_{order_cap}.txt"

# determine simulation times in units of the OAT strength \chi
time_steps = 10**3 # time steps in plot
max_tau = 1 # for simulation: chi * max_time = max_tau * N **(-2/3)
tau_vals = np.linspace(0, max_tau, time_steps)
times = tau_vals * N**(-2/3)

ivp_tolerance = 1e-10 # relative error tolerance in numerical integrator

S = N/2
h_zxy = {}
h_zxy[NAT] = {}
h_zxy[OAT] = { (2,0,0) : 1 }
h_zxy[TAT] = { (2,0,0) : +1/3,
               (0,0,2) : -1/3 }
h_zxy[TNT] = { (2,0,0) : 1,
               (0,1,0) : S }
h_zxy[TNB] = { (2,0,0) : 1,
                (0,1,0) : -S }

init_state_X = (0,1,0)
init_state_nZ = (-1,0,0)
mat_X_to_nZ_zxy = -np.array([ [0,1,0], [1,0,0], [0,0,1] ])
mat_X_to_nZ_pzm = mat_zxy_to_pzm(mat_X_to_nZ_zxy)

dec_names = [ no_dec, dec_M, dec_m, dec_P, dec_p, dec_Z, dec_z ]

collective_dec = 1/3
single_dec = collective_dec * S
dec_rates = { no_dec : None,
              dec_p : (single_dec,0,0),
              dec_z : (0,single_dec,0),
              dec_m : (0,0,single_dec),
              dec_P : ( (0,0,0), (collective_dec,0,0) ),
              dec_Z : ( (0,0,0), (0,collective_dec,0) ),
              dec_M : ( (0,0,0), (0,0,collective_dec) ) }
titles = { no_dec : "No decoherence",
           dec_M : r"$\Gamma_-=\chi/3$",
           dec_m : r"$\gamma_-=S\chi/3$",
           dec_P : r"$\Gamma_+=\chi/3$",
           dec_p : r"$\gamma_+=S\chi/3$",
           dec_Z : r"$\Gamma_{\mathrm{z}}=\chi/3$",
           dec_z : r"$\gamma_{\mathrm{z}}=S\chi/3$" }
titles[dicke] = titles[no_dec]
for dec in dec_names:
    for single_dict in all_dicts: single_dict[dec] = {}

A_zxy = { (0,1,0) : 1,
          (0,0,1) : 1j }
B_zxy = { (0,1,0) : 1,
          (0,0,1) : -1j }

In [4]:
##########################################################################################
# Dicke manifold simulation methods
##########################################################################################

II = scipy.sparse.identity(N+1)
S_op_vec, _ = spin_op_vec_mat_dicke(N)

def vec_to_dicke(op_vec_zxy):
    op_dicke = 0 * II
    for op, val in op_vec_zxy.items():
        op_dicke_term = II.copy()
        for mm in range(3):
            for ii in range(op[mm]):
                op_dicke_term = op_dicke_term @ S_op_vec[mm]
        op_dicke += val * op_dicke_term
    return op_dicke

A_dicke = vec_to_dicke(A_zxy)
B_dicke = vec_to_dicke(B_zxy)
H = { method : vec_to_dicke(h_zxy[method]) for method in methods }

init_state_dicke = coherent_spin_state(init_state_X, N)
def get_states(hamiltonian, extra_op = II):
    return solve_ivp(lambda time, state : -1j * ( hamiltonian @ state ),
                     (0,times[-1]), extra_op @ init_state_dicke, t_eval = times,
                     rtol = ivp_tolerance, atol = ivp_tolerance).y

def corr_vals(op_mat, method, prepend_op = II, append_op = II):
    kets = get_states(H[method], append_op)
    if (prepend_op != append_op).nnz == 0: # if prepend_op == append_op
        bras = kets
    else:
        bras = get_states(H[method], prepend_op.conj().T)
    return np.array([ bras[:,tt].conj() @ ( op_mat @ kets[:,tt] )
                      for tt in range(times.size) ])

##########################################################################################
# full simulation methods
##########################################################################################

init_state = { method : init_state_X for method in methods }
dec_mat = { method : None for method in methods }
h_pzm = { method : vec_zxy_to_pzm(h_zxy[method]) for method in methods }
A_pzm = { method : vec_zxy_to_pzm(A_zxy) for method in methods }
B_pzm = { method : vec_zxy_to_pzm(B_zxy) for method in methods }

for method in methods:
    if N < 10**4 or method in [ NAT, OAT ]: continue
    init_state[method] = init_state_nZ
    dec_mat[method] = mat_X_to_nZ_pzm
    h_pzm[method] = vec_zxy_to_pzm(h_zxy[method], init_state_X)
    A_pzm[method] = vec_zxy_to_pzm(A_zxy, init_state_X)
    B_pzm[method] = vec_zxy_to_pzm(B_zxy, init_state_X)

def compute_deriv_op_vecs(op, dec, method):
    return get_deriv_op_vec(order_cap, N, init_state[method], h_pzm[method],
                            dec_rates[dec], dec_mat[method], list(op.keys()), True)

In [5]:
def save_vec(vec, save_file):
    with open(save_file, "w") as f:
        for key in vec.keys():
            f.write(f"key {key}\n")
            for op, val in vec[key].items():
                f.write(f"{op} {val}\n")

def load_vec(load_file):
    vec = {}
    with open(load_file, "r") as f:
        for line in f:
            line_items = line.split()
            if line_items[0] == "key":
                key = eval(" ".join(line_items[1:]))
                vec[key] = {}
            else:
                op = eval(" ".join(line_items[:-1]))
                val = eval(line_items[-1])
                vec[key][op] = val
    return vec

In [6]:
if N <= 10**3:

    start = time()
    for single_dict in all_dicts: single_dict[dicke] = {}
    for method in methods:
        print(method)
        corrs_A[dicke][method] = corr_vals(A_dicke, method)
        corrs_B[dicke][method] = corr_vals(B_dicke, method)
        corrs_At_B[dicke][method] = corr_vals(A_dicke, method, append_op = B_dicke)
        corrs_B_At[dicke][method] = corr_vals(A_dicke, method, prepend_op = B_dicke)
    print(time()-start)

In [7]:
start = time()

for dec in dec_names:
    print(dec)
    for method in methods:
        print("",method)

        if os.path.isfile(save_file_name(vecs_A_txt, dec, method)):
            vecs_A[dec][method] = load_vec(save_file_name(vecs_A_txt, dec, method))
        else:
            vecs_A[dec][method] = compute_deriv_op_vecs(A_pzm[method], dec, method)
            save_vec(vecs_A[dec][method], save_file_name(vecs_A_txt, dec, method))

        if os.path.isfile(save_file_name(vecs_B_txt, dec, method)):
            vecs_B[dec][method] = load_vec(save_file_name(vecs_B_txt, dec, method))
        else:
            vecs_B[dec][method] = compute_deriv_op_vecs(B_pzm[method], dec, method)
            save_vec(vecs_B[dec][method], save_file_name(vecs_B_txt, dec, method))

        if os.path.isfile(save_file_name(vecs_At_B_txt, dec, method)):
            vecs_At_B[dec][method] = load_vec(save_file_name(vecs_At_B_txt, dec, method))
        else:
            vecs_At_B[dec][method] = sandwich_deriv_op_vec(vecs_A[dec][method],
                                                           append_op = B_pzm[method])
            save_vec(vecs_At_B[dec][method], save_file_name(vecs_At_B_txt, dec, method))

        if os.path.isfile(save_file_name(vecs_B_At_txt, dec, method)):
            vecs_B_At[dec][method] = load_vec(save_file_name(vecs_B_At_txt, dec, method))
        else:
            vecs_B_At[dec][method] = sandwich_deriv_op_vec(vecs_A[dec][method],
                                                           prepend_op = B_pzm[method])
            save_vec(vecs_B_At[dec][method], save_file_name(vecs_B_At_txt, dec, method))

        vecs_At_B_comm[dec][method] \
            = add_deriv_op_vecs(vecs_At_B[dec][method],
                                vecs_B_At[dec][method], factor_rht = -1)

        deriv_vals_A = deriv_op_vec_to_vals(vecs_A[dec][method], N, init_state[method])
        deriv_vals_B = deriv_op_vec_to_vals(vecs_B[dec][method], N, init_state[method])
        deriv_vals_At_B = deriv_op_vec_to_vals(vecs_At_B[dec][method], N, init_state[method])
        deriv_vals_B_At = deriv_op_vec_to_vals(vecs_B_At[dec][method], N, init_state[method])

        all_corrs_A = deriv_vals_to_correlators(deriv_vals_A, times)
        all_corrs_B = deriv_vals_to_correlators(deriv_vals_B, times)
        all_corrs_At_B = deriv_vals_to_correlators(deriv_vals_At_B, times)
        all_corrs_B_At = deriv_vals_to_correlators(deriv_vals_B_At, times)

        corrs_A[dec][method] \
            = sum( val * all_corrs_A[key] for key, val in A_pzm[method].items() )
        corrs_B[dec][method] \
            = sum( val * all_corrs_B[key] for key, val in B_pzm[method].items() )
        corrs_At_B[dec][method] \
            = sum( val * all_corrs_At_B[key] for key, val in A_pzm[method].items() )
        corrs_B_At[dec][method] \
            = sum( val * all_corrs_B_At[key] for key, val in A_pzm[method].items() )

for key in corrs_A.keys():
    for method in methods:
        corrs_At_B_disc[key][method] \
         = corrs_At_B[key][method] - corrs_A[key][method] * corrs_B[key][method][0]
        corrs_B_At_disc[key][method] \
         = corrs_B_At[key][method] - corrs_A[key][method] * corrs_B[key][method][0]

print(time()-start)

no_dec
 OAT


 TAT


 TNT


dec_M
 OAT


 TAT


...

dec_Z
 OAT
 TAT


 TNT


dec_z
 OAT
 TAT


 TNT


537.330160856247


In [8]:
if order_cap <= 20:

    start = time()
    for dec in dec_names:
        print(dec)
        for method in methods:
            print("",method)

            if os.path.isfile(save_file_name(vecs_At_B_comm_sqr_txt, dec, method)):
                vecs_At_B_comm_sqr[dec][method] \
                    = load_vec(save_file_name(vecs_At_B_comm_sqr_txt, dec, method))
            else:
                vecs_At_B_comm_sqr[dec][method] \
                    = multiply_deriv_op_vecs(vecs_At_B_comm[dec][method],
                                             vecs_At_B_comm[dec][method],
                                             dag_lft = True)
                save_vec(vecs_At_B_comm_sqr[dec][method],
                         save_file_name(vecs_At_B_comm_sqr_txt, dec, method))
    print(time()-start)

In [9]:
fontsize = 9
max_time = 3.5
figsize = (7,4.5)
mag_phs_ratio = 2
legend_y = -1.5
plot_methods = [ OAT, TAT, TNT ]

dec_grid_map = {}
dec_grid_map[no_dec] = (0,-1)
dec_grid_map[dicke] = (1,-1)
for col, dec in enumerate(dec_names[1::2]):
    dec_grid_map[dec] = (0,col)
for col, dec in enumerate(dec_names[2::2]):
    dec_grid_map[dec] = (1,col)

mag = "mag" # key for correlator magnitude
phs = "phs" # key for correlator phase

params = { "text.usetex" : True,
           "font.size" : fontsize,
           "axes.titlesize" : fontsize,
           "axes.labelsize" : fontsize }
plt.rcParams.update(params)
plot_idx = N * times <= max_time

plot_methods = [ method for method in plot_methods if method in methods ]
colors = [ "#4E79A7", "#F28E2B", "#E15759", "#76B7B2",
           "#59A14E", "#EDC949", "#B07AA2", "#FF9DA7",
           "#9C755F", "#BAB0AC" ]

def plot_corrs(corrs, filename, ylabels):
    plot_dicke = dicke in corrs.keys()
    dec_columns = len(corrs.keys()) // 2
    height_ratios = [ mag_phs_ratio, 1 ]

    axes = { dec : {} for dec in corrs.keys() }

    plt.figure(figsize = figsize)
    full_grid = gridspec.GridSpec(1, 2, width_ratios = [ 1, dec_columns ])
    if plot_dicke:
        benchmarking_grid = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec = full_grid[0])
        subgrids = {}
        subgrids[0] = gridspec.GridSpecFromSubplotSpec(2, 1, height_ratios = height_ratios,
                                                       subplot_spec = benchmarking_grid[0])
        subgrids[1] = gridspec.GridSpecFromSubplotSpec(2, 1, height_ratios = height_ratios,
                                                       subplot_spec = benchmarking_grid[1])

        axes[no_dec][mag] = plt.subplot(subgrids[dec_grid_map[no_dec][0]][0])
        axes[no_dec][phs] = plt.subplot(subgrids[dec_grid_map[no_dec][0]][1])
        axes[dicke][mag] = plt.subplot(subgrids[dec_grid_map[dicke][0]][0],
                                       sharex = axes[no_dec][mag], sharey = axes[no_dec][mag])
        axes[dicke][phs] = plt.subplot(subgrids[dec_grid_map[dicke][0]][1],
                                       sharex = axes[no_dec][mag], sharey = axes[no_dec][phs])
    else:
        mid_height_ratios = [ (1+mag_phs_ratio)/2, mag_phs_ratio, 1, (1+mag_phs_ratio)/2 ]
        benchmarking_grid \
            = gridspec.GridSpecFromSubplotSpec(4, 1, height_ratios = mid_height_ratios,
                                               subplot_spec = full_grid[0])
        axes[no_dec][mag] = plt.subplot(benchmarking_grid[1])
        axes[no_dec][phs] = plt.subplot(benchmarking_grid[2])

    dec_grid = gridspec.GridSpecFromSubplotSpec(2, dec_columns, subplot_spec = full_grid[1])
    for dec in corrs.keys():
        if dec in [ no_dec, dicke ]: continue
        subgrid = gridspec.GridSpecFromSubplotSpec(2, 1, height_ratios = height_ratios,
                                                   subplot_spec = dec_grid[dec_grid_map[dec]])
        axes[dec][mag] = plt.subplot(subgrid[0], sharex = axes[no_dec][mag],
                                     sharey = axes[no_dec][mag])
        axes[dec][phs] = plt.subplot(subgrid[1], sharex = axes[no_dec][mag],
                                     sharey = axes[no_dec][phs])

    for dec, corr in corrs.items():
        axis = axes[dec]
        axis[mag].set_title(titles[dec])

        for method, color in zip(plot_methods, colors):
            axis[mag].plot(N*times[plot_idx], abs(corr[method][plot_idx])/S, color = color)
            axis[phs].plot(N*times[plot_idx], np.angle(corr[method][plot_idx])/np.pi,
                           color = color)

        plt.setp(axis[mag].get_xticklabels(), visible = False)
        if dec_grid_map[dec][0] == 1:
            axis[phs].set_xlabel(r"Time ($N\chi t$)")
        else:
            plt.setp(axis[phs].get_xticklabels(), visible = False)

        for val in [ mag, phs ]:
            if dec in [ no_dec, dicke ]:
                axis[val].set_ylabel(ylabels[val])
            else:
                plt.setp(axis[val].get_yticklabels(), visible = False)
                axis[val].yaxis.get_offset_text().set_visible(False)

            axis[val].tick_params(direction = "in")
            axis[val].grid(True)

    color_handles = [ Line2D([0], [0], color = color) for color in colors ]
    if plot_dicke:
        axes[no_dec][mag].legend(color_handles, plot_methods, loc = "upper left")
    else:
        axes[no_dec][phs].legend(color_handles, plot_methods, loc = "lower center",
                                 bbox_to_anchor = (0.5, legend_y))

    plt.tight_layout()
    plt.savefig(filename)

In [10]:
%matplotlib agg
plot_corrs(corrs_At_B_disc, fig_dir + "two_time.pdf",
           { mag : r"$\left|C(t)\right|$", phs : r"$\phi(t)/\pi$" })